In [1]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime
import pickle
from geopy import distance
import numpy as np
import time
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from os import listdir

In [2]:
start = time.time()

dict_dfs={}

dict_dfs={"grad":["nb39-jx2v"], "trees":["uvpi-gqnh"],"crashes":["h9gi-nx95"],"pc":["cuzb-dmcd"],
          "shootings":["833y-fsy8"], "arrests":["8h9b-rp9u"],"programs":["mbd7-jfnc"],"bins":["sxx4-xhzg"]}
for key in dict_dfs:
    x=dict_dfs[key][0]
    dict_dfs[key]={"key":x,"last_updated":0,"data":""}
    
dict_dfs

{'grad': {'key': 'nb39-jx2v', 'last_updated': 0, 'data': ''},
 'trees': {'key': 'uvpi-gqnh', 'last_updated': 0, 'data': ''},
 'crashes': {'key': 'h9gi-nx95', 'last_updated': 0, 'data': ''},
 'pc': {'key': 'cuzb-dmcd', 'last_updated': 0, 'data': ''},
 'shootings': {'key': '833y-fsy8', 'last_updated': 0, 'data': ''},
 'arrests': {'key': '8h9b-rp9u', 'last_updated': 0, 'data': ''},
 'programs': {'key': 'mbd7-jfnc', 'last_updated': 0, 'data': ''},
 'bins': {'key': 'sxx4-xhzg', 'last_updated': 0, 'data': ''}}

In [4]:
def get_avgGradRate(df_school, df_scatter):
    df=df_school
    df1=df_scatter
    df["total_grads_of_cohort"]=df["total_grads_of_cohort"].apply(float)
    x=df.groupby(["dbn"]).mean()
    x=x.reset_index()
    x=x.rename(columns={"total_grads_of_cohort":"avg_perc Grads"})
    x=x[["dbn","avg_perc Grads"]]
    df1=df1.merge(x, on= "dbn", how="left")
    return df1

def clean_school(df_grad):
    df=df_grad
    df= df[(df["demographic_variable"]=="All Students")&(df["demographic_category"]=="All Students")]
    print(df.columns)
    df=df[["dbn","cohort_year","cohort","total_grads_of_cohort"]]
    df=df[df.total_grads_of_cohort.notnull()]
    return df





In [ ]:
def get_dataframes(data_dict):
    data_url = "https://data.cityofnewyork.us/resource/"
    limit=".json?$limit=1000000000&$$app_token=HnDwBjZUCabVQojpbl6aIDrDI"
    token="&$$app_token=HnDwBjZUCabVQojpbl6aIDrDI"
    for key in data_dict:
        print(key)
        if key in ["addresses","scatter"]:
            continue
        
        t1=time.time()
        
        if key == "grad":
            limit=".json?$limit=1000000&$$app_token=HnDwBjZUCabVQojpbl6aIDrDI"
            url=data_url+data_dict[key]["key"]+limit
            df=pd.read_json(url)
            df.to_csv("datasets/dataframes/grad.csv")
            data_dict[key]["df"]=df
            
        else:
            limit=".json?$limit=10000"
            o=0
            while True:
                off=str(o)
                off_set="&$offset="+off
                url=data_url+data_dict[key]["key"]+limit+off_set+token
                df=pd.read_json(url)
                if "lat" in df.columns:
                    df=df.rename(columns={"lat":"latitude","lon":"longitude"})
                try:
                    df=df[["latitude","longitude"]]

                    df=df[df.latitude.notna()]
                    df=df[df.longitude.notna()]
                    df["latitude"]=df["latitude"].apply(float)
                    df["longitude"]=df["longitude"].apply(float)
                    df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
                except:
                    print("except")
                    break
                file="datasets/dataframes/"+key+str(off)+".csv"
                df.to_csv(file)
                
                o=o+10000
                print(o,"    ", ((time.time()-t1)/60))
                
        end=time.time()
        print(key, ((end-t1)/60))
        
        #data_dict[key]["df"]=df
        print(key + " done")
    df=pd.read_csv("datasets/2016_DOE_High_School_Directory-2.csv")
    #df["location"]=list(zip(df["Latitude"].to_list(),df["Longitude"].to_list()))
    df=df.rename(columns={"Latitude":"latitude","Longitude":"longitude"})
    df=df[["dbn","school_name","borough","latitude","longitude"]]
    df["latitude"]=df["latitude"].apply(float)
    df["longitude"]=df["longitude"].apply(float)
    df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
    data_dict["grad"]["df"]=clean_school(dict_dfs["grad"]["df"])
    df=get_avgGradRate(data_dict["grad"]["df"], df)
    #df=df.drop(columns=[""])
    x=df[["dbn","latitude","longitude","location"]]
    data_dict["grad"]["df"]=data_dict["grad"]["df"].merge(x,on="dbn",how="left")
    data_dict["scatter"]={"df":df}
    df.to_csv("datasets/dataframes/scatter.csv")
    
get_dataframes(dict_dfs)


    
print("since start",(time.time()-start)/60)


In [5]:
def get_scatterGrad(data_dict):
    df=pd.read_csv("datasets/2016_DOE_High_School_Directory-2.csv")
    #df["location"]=list(zip(df["Latitude"].to_list(),df["Longitude"].to_list()))
    df=df.rename(columns={"Latitude":"latitude","Longitude":"longitude"})
    df=df[["dbn","school_name","borough","latitude","longitude"]]
    df["latitude"]=df["latitude"].apply(float)
    df["longitude"]=df["longitude"].apply(float)
    df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
    data_dict["grad"]["df"]=pd.read_csv("datasets/dataframes/grad.csv")
    data_dict["grad"]["df"]=clean_school(dict_dfs["grad"]["df"])
    df=get_avgGradRate(data_dict["grad"]["df"], df)
    x=df[["dbn","latitude","longitude","location"]]
    data_dict["grad"]["df"]=data_dict["grad"]["df"].merge(x,on="dbn",how="left")
    data_dict["scatter"]={"df":df}
    df.to_csv("datasets/dataframes/scatter.csv")
    
get_scatterGrad(dict_dfs)

Index(['Unnamed: 0', 'dbn', 'school_name', 'demographic_category',
       'demographic_variable', 'cohort_year', 'cohort', 'total_cohort',
       'total_grads', 'total_grads_of_cohort', 'total_regents',
       'total_regents_of_cohort', 'total_regents_of_grads', 'advanced_regents',
       'advanced_regents_of_cohort', 'advanced_regents_of_grads',
       'regents_w_o_advanced', 'regents_w_o_advanced_of_cohort',
       'regents_w_o_advanced_of_grads', 'local', 'local_of_cohort',
       'local_of_grads', 'still_enrolled', 'still_enrolled_of_cohort',
       'dropped_out', 'dropped_out_of_cohort'],
      dtype='object')


In [6]:
def get_radLocList(df):
    df["latitude_rad"]=df["latitude"]
    df["longitude_rad"]=df["longitude"]
    df["latitude_rad"]=df["latitude_rad"].apply(lambda x:radians(x))
    df["longitude_rad"]=df["longitude_rad"].apply(lambda x:radians(x))
    l=list(zip(df["latitude_rad"].to_list(),df["longitude_rad"].to_list()))
    return l
    
    
def getFeat(l1,l2):
    res=haversine_distances(l1,l2)
    res=res * 6371000/1000
    res1=res < 0.5
    feat= np.sum(res1, axis=1)
    return feat

def allFeat(data_dict):
    t1=time.time()
    df_scatter=data_dict["scatter"]["df"]
    lScatter=get_radLocList(df_scatter)
    files=listdir("datasets/dataframes")
    for key in data_dict:
        t2=time.time()
        
        if ((key != "grad") & (key != "scatter")):
            df_scatter[key]=0
            print(key)
            for file in files:
                if key in file:
                    path="datasets/dataframes/"+file
                    df=pd.read_csv(path)
                    l_feat=get_radLocList(df)
                    feat=getFeat(lScatter,l_feat)
                    df_scatter[key]=df_scatter[key]+feat
                    print(file + "done")
        print(key, " done ",(time.time()-t2)/60)
    df_scatter.drop(columns=["latitude_rad","longitude_rad"])
    #data_dict["scatter"]["df"]=df_scatter
    print("all done", (time.time()-t1)/60)
    return df_scatter

x=allFeat(dict_dfs)
print("since start",(time.time()-start)/60)
x.to_csv("datasets/dataframes/scatter.csv")
x.to_csv("datasets/dataForScatter.csv")

grad  done  1.5894571940104166e-08
trees
trees50000.csvdone
trees610000.csvdone
trees130000.csvdone
trees350000.csvdone
trees250000.csvdone
trees570000.csvdone
trees470000.csvdone
trees680000.csvdone
trees240000.csvdone
trees340000.csvdone
trees460000.csvdone
trees560000.csvdone
trees40000.csvdone
trees600000.csvdone
trees120000.csvdone
trees540000.csvdone
trees440000.csvdone
trees360000.csvdone
trees260000.csvdone
trees190000.csvdone
trees60000.csvdone
trees100000.csvdone
trees620000.csvdone
trees70000.csvdone
trees110000.csvdone
trees630000.csvdone
trees450000.csvdone
trees550000.csvdone
trees270000.csvdone
trees370000.csvdone
trees0.csvdone
trees180000.csvdone
trees500000.csvdone
trees400000.csvdone
trees320000.csvdone
trees220000.csvdone
trees140000.csvdone
trees660000.csvdone
trees20000.csvdone
trees490000.csvdone
trees590000.csvdone
trees150000.csvdone
trees670000.csvdone
trees30000.csvdone
trees580000.csvdone
trees480000.csvdone
trees410000.csvdone
trees510000.csvdone
trees23000

arrests3030000.csvdone
arrests4870000.csvdone
arrests4970000.csvdone
arrests150000.csvdone
arrests2360000.csvdone
arrests2260000.csvdone
arrests4630000.csvdone
arrests4730000.csvdone
arrests1500000.csvdone
arrests1400000.csvdone
arrests480000.csvdone
arrests580000.csvdone
arrests3950000.csvdone
arrests3850000.csvdone
arrests1320000.csvdone
arrests1220000.csvdone
arrests4010000.csvdone
arrests4110000.csvdone
arrests930000.csvdone
arrests2540000.csvdone
arrests2440000.csvdone
arrests830000.csvdone
arrests4570000.csvdone
arrests4470000.csvdone
arrests2020000.csvdone
arrests2120000.csvdone
arrests1640000.csvdone
arrests1740000.csvdone
arrests1060000.csvdone
arrests1160000.csvdone
arrests2600000.csvdone
arrests3180000.csvdone
arrests2700000.csvdone
arrests3080000.csvdone
arrests4350000.csvdone
arrests4250000.csvdone
arrests4680000.csvdone
arrests5100000.csvdone
arrests4780000.csvdone
arrests5000000.csvdone
arrests3450000.csvdone
arrests3550000.csvdone
arrests2840000.csvdone
arrests430000.cs

arrests550000.csvdone
arrests2920000.csvdone
arrests2820000.csvdone
arrests450000.csvdone
arrests5060000.csvdone
arrests3530000.csvdone
arrests3430000.csvdone
arrests180000.csvdone
arrests1100000.csvdone
arrests1000000.csvdone
arrests2760000.csvdone
arrests2660000.csvdone
arrests4230000.csvdone
arrests4330000.csvdone
arrests4410000.csvdone
arrests4510000.csvdone
arrests2140000.csvdone
arrests2040000.csvdone
arrests1720000.csvdone
arrests1620000.csvdone
arrests40000.csvdone
arrests1540000.csvdone
arrests1440000.csvdone
arrests4670000.csvdone
arrests4770000.csvdone
arrests2320000.csvdone
arrests2220000.csvdone
arrests3280000.csvdone
arrests2500000.csvdone
arrests970000.csvdone
arrests3380000.csvdone
arrests870000.csvdone
arrests2400000.csvdone
arrests4050000.csvdone
arrests4150000.csvdone
arrests1360000.csvdone
arrests1260000.csvdone
arrests3910000.csvdone
arrests3810000.csvdone
arrests4580000.csvdone
arrests4480000.csvdone
arrests3750000.csvdone
arrests3650000.csvdone
arrests730000.csvd

In [ ]:
x.isnull().sum()

In [7]:
df_scatter=pd.read_csv("datasets/dataForScatter.csv")

def featuresForSchools(df_scatter, df_grad):
    df= df_grad.merge(df_scatter, on="dbn",how="left")
    df= df.drop(columns=["Unnamed: 0"])
    return df

x=featuresForSchools(df_scatter,dict_dfs["grad"]["df"])

In [8]:
def normalize_df(df, columns):
    for col in columns:
        df[col]=((df[col]-df[col].mean())/df[col].std())
    return df

x_norm=normalize_df(x,["trees","crashes", "pc", "shootings", "arrests","programs","bins"])
x_norm.to_csv("datasets/dataForML.csv")

In [ ]:
end2=time.time()

print("/n/n",(end2-start)/59.992)

In [ ]:
def get_data(data_dict):
    data_url = "https://data.cityofnewyork.us/resource/"
    token = "kqG7loCHxQI54gSuyKyXgagTL"
    client = Socrata(data_url,token, timeout=12000)
    for key in data_dict:
        data_set=data_dict[key]["key"]
        meta = client.get_metadata(data_set)
        print(meta)
        if meta["viewLastModified"]>data_dict[key]["last_updated"]:
            data_dict[key]["last_updated"]=meta["viewLastModified"]
            data_dict[key]["data"]=client.get(data_set,limit=1000)
            lastUpdated=True
        print(key + " done")      
    
     

get_data(dict_dfs)
print("since start",(time.time()-start)/60)